In [6]:
!pip install LughaatNLP

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 77.8 MB/s eta 0:00:00


In [93]:
import pandas as pd
from google.colab import files
from LughaatNLP import LughaatNLP
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


**Loading the dataset**

In [31]:
uploaded=files.upload()

Saving urdu_sarcastic_dataset.csv to urdu_sarcastic_dataset (3).csv


In [269]:
# Load the entire dataset
df = pd.read_csv('urdu_sarcastic_dataset.csv')

# Drop columns where the header is 'Unnamed:'
data = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Display the first few rows of the sampled DataFrame
data.head()


,urdu_text,is_sarcastic
0,🤣😂😂 ہو لینے دے میری شادی فسادن ٹھیک ہے کوجی نہیں چاہیے 😐😐😐🤣,1.0
1,چل مہمانوں میں کھانا سرو کر چڑیل چاچی نوں دسدی آں میں😂😂,1.0
2,کامران خان آپکی دن بھریہ زمہ داری لگائی گئی اپوزیشن کی کردار کشی اوراس پربھونکناہےآپ خوشامدگری وچاپلوسی سےاورکتنی دولت کماناچاہتےہیں موٹرسائیکل سےپیجارو پراڈو تک کےسفرمیں ضمیرکی لاش سےاٹھتی بدبوآپ کی ناک بند نہیں کرتی ہے 🙏نوٹ آپ سب سےالتجاگزارش ہےہمیں بھی فالوکریں شکریہ,0.0
3,نہیں پائین 😎,0.0
4,`` مراد علی شاہ کے بھیس میں ڈی جی آئی ایس آئی تھے '' حامد میر😁,1.0


In [270]:
# Set pandas to display the full content of each row
pd.set_option('display.max_colwidth', None)  # This ensures that column contents are displayed fully
pd.set_option('display.max_rows', None)      # This shows all rows (if needed)
pd.set_option('display.expand_frame_repr', False)  # Prevent line breaks

In [271]:
# Initialize LughaatNLP instance
urdu_text_processing = LughaatNLP()


In [272]:
original_data=data.copy()  #creating a copy so we can backup where needed

**Preprocessing**(Removing Stopword
Punctuation, Emojis, and Hashtags
Short Conversations etc)

In [273]:
#preprocessing
# Define a function to apply the preprocessing steps
def preprocess_text(text):
    # Ensure the input is a string before processing
    if isinstance(text, str):

        # Step 1: Remove diacritics
        text = urdu_text_processing.remove_diacritics(text)

        # Step 2: Remove punctuation and extra spaces
        text = urdu_text_processing.punctuations_space(text)

        # Step 3: Remove URLs
        text = urdu_text_processing.remove_urls(text)

        # Step 4: Remove emojis (assuming emojis are considered special characters in Urdu)
        text = urdu_text_processing.remove_special_characters(text)

        # Step 5: Remove hashtags
        text = text.replace('#', '')

        # Step 6: Remove short conversational phrases (you may set a threshold like < 3 words)
        if len(text.split()) < 3:  # Remove if too short (less than 3 words)
            return ''

        return text
    return text  # If it's not a string, return as is

In [274]:
urdu_stopwords = [
    "آ", "آئی", "آئیں", "آئے", "آتا", "آتی", "آتے", "آداب", "آدھ", "آدھا", "آدھی", "آدھے", "آس", "آمدید", "آنا", "آنسہ",
    "آنی", "آنے", "آپ", "آگے", "آہ", "آہا", "آیا", "اب", "ابھی", "ابے", "اتوار", "ارب", "اربویں", "ارے", "اس", "اسکا",
    "اسکی", "اسکے", "اسی", "اسے", "اف", "افوہ", "الاول", "البتہ", "الثانی", "الحرام", "السلام", "الف", "المکرم",
    "ان", "اندر", "انکا", "انکی", "انکے", "انہوں", "انہی", "انہیں", "اوئے", "اور", "اوپر", "اوہو", "اپ", "اپنا", "اپنوں",
    "اپنی", "اپنے", "اپنےآپ", "اکبر", "اکثر", "اگر", "اگرچہ", "اگست", "اہاہا", "ایسا", "ایسی", "ایسے", "ایک", "بائیں",
    "بار", "بارے", "بالکل", "بے", "بغیر", "بلکہ", "بن", "بنا", "بناؤ", "بند", "بڑی", "بھر", "بھریں", "بھی", "بہار",
    "بہت", "بہتر", "بیگم", "تاکہ", "تاہم", "تب", "تجھ", "تجھھی", "تجھے", "ترا", "تری", "تلک", "تم", "تمام", "تمہارا",
    "تمہاروں", "تمہاری", "تمہارے", "تمہیں", "تو", "تک", "تھا", "تھی", "تھیں", "تھے", "تہائی", "تیرا", "تیری", "تیرے",
    "تین", "جا", "جاؤ", "جائیں", "جائے", "جاتا", "جاتی", "جاتے", "جانی", "جانے", "جب", "جبکہ", "جدھر", "جس", "جسے",
    "جن", "جناب", "جنہوں", "جنہیں", "جو", "کہاں", "کہہ", "کہی", "کہیں", "کہے", "کی", "کیا", "کیسا", "کیسے", "کیونکر",
    "کیوں", "کیے", "کے", "گئی", "گئے", "گا", "گرما", "گرمی", "گنا", "گو", "گویا", "گھنٹا", "گھنٹوں", "گھنٹے", "گی",
    "گیا", "ہائیں", "ہائے", "ہاڑ", "ہاں", "ہر", "ہرچند", "ہرگز", "ہزار", "ہفتہ", "ہم", "ہمارا", "ہماری", "ہمارے", "ہمی",
    "ہمیں", "ہو", "ہوئی", "ہوئیں", "ہوئے", "ہوا", "ہوبہو", "ہوتا", "ہوتی", "ہوتیں", "ہوتے", "ہونا", "ہونگے", "ہونی",
    "ہونے", "ہوں", "ہی", "ہیلو", "ہیں", "ہے", "یا", "یات", "یعنی", "یک", "یہ", "یہاں", "یہی", "یہیں", "کر",
    "میری", "میں",
    "کو", "سے", "کا", "نے"
]

In [275]:
# Function to remove stop words
def remove_stop_words(text):
    words = text.split()  # Split the text into words
    cleaned_words = [word for word in words if word not in urdu_stopwords]  # Filter out stop words
    return ' '.join(cleaned_words)  # Join the cleaned words back into a string


In [276]:
#drop rows if data missing or empty string
data = data.dropna(subset=['urdu_text']) # for missing
data['urdu_text'] = data['urdu_text'].str.strip()  # Remove leading and trailing whitespace
data = data[data['urdu_text'] != '']  # Drop empty strings

# Apply the preprocessing function to the 'urdu_text' column
data['urdu_text'] = data['urdu_text'].apply(preprocess_text)

#remove stopwords now
data['clean_text'] = data['urdu_text'].apply(remove_stop_words)

<ipython-input-276-4a2b7a13fa63>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['urdu_text'] = data['urdu_text'].str.strip()  # Remove leading and trailing whitespace


**Clearing after the preprocessing**

In [277]:
# Strip whitespace and drop empty strings
data['clean_text'] = data['clean_text'].str.strip()  # Remove leading and trailing whitespace
data = data[data['clean_text'] != '']  # Drop empty strings
data['clean_text'].head()

,clean_text
0,لینے دے شادی فسادن ٹھیک کوجی نہیں چاہیے
1,چل مہمانوں کھانا سرو چڑیل چاچی نوں دسدی اں
2,کامران خان اپکی دن بھریہ زمہ داری لگائی اپوزیشن کردار کشی اوراس پربھونکناہےاپ خوشامدگری وچاپلوسی سےاورکتنی دولت کماناچاہتےہیں موٹرسائیکل سےپیجارو پراڈو کےسفرمیں ضمیرکی لاش سےاٹھتی بدبواپ ناک نہیں کرتی نوٹ سب سےالتجاگزارش ہےہمیں فالوکریں شکریہ
4,مراد علی شاہ بھیس ڈی جی ائی ایس ائی حامد میر
5,قابل اعتبار قاتل اعتبار


**Stemming**

In [278]:
#stemming
data['stemmed_text'] = data['clean_text'].apply(urdu_text_processing.urdu_stemmer)

In [279]:
data['stemmed_text'].head()

,stemmed_text
0,لینہ دہ شادی فسادن ٹھیک کوجی نہا چاہیہ
1,چل مہمانا کھانا سرو چڑیل چاچی نا دسدی اں
2,کامران خان اپکی دن بھریہ زمہ داری لگائی اپوزیشن کردار کشی اوراس پربھونکناہےاپ خوشامدگری وچاپلوسی سےاورکتنی دولت کماناچاہتےہا موٹرسائیکل سےپیجارو پراڈو کےسفرما ضمیرکی لاش سےاٹھتی بدبواپ ناک نہا کرتی نوٹ سب سےالتجاگزارش ہےہما فالوکرا شکریہ
4,مراد علی شاہ بھیس ڈی جی ائی ایس ائی حامد میر
5,قابل اعتبر قاتل اعتبر


**Lemmatization**

In [280]:
#lemmatization
data['lemmatized_text'] = data['clean_text'].apply(urdu_text_processing.lemmatize_sentence)

In [281]:
data['lemmatized_text'].head()

,lemmatized_text
0,لینا دینا شادی فسادن ٹھیک کوجی نہیں چاہنا
1,چلنا مہمان کھا سرا چڑیل چاچی نوں دسدی اں
2,کامران خان اپکی دن بھریہ زمہ داری لگنا اپوزیشن کردار کشی اوراس پربھونکناہےاپ خوشامدگری وچاپلوسی سےاورکتنی دولت کماناچاہتےہیں موٹرسائیکل سےپیجارو پراڈو کےسفرمیں ضمیرکی لاش سےاٹھتی بدبواپ ناک نہیں کرنا نوٹ سب سےالتجاگزارش ہےہمیں فالوکریں شکریہ
4,مراد علی شاہ بھیس ڈی جینا ائی ایس ائی حامد میر
5,قابل اعتبار قاتل اعتبار


**Feature Extraction from Urdu**

Tokenization

In [282]:
data['tokenized_text'] = data['clean_text'].apply(urdu_text_processing.urdu_tokenize)

In [283]:
data['tokenized_text'].head()

,tokenized_text
0,"[لینے, دے, شادی, فسادن, ٹھیک, کوجی, نہیں, چاہیے]"
1,"[چل, مہمانوں, کھانا, سرو, چڑیل, چاچی, نوں, دسدی, اں]"
2,"[کامران, خان, اپکی, دن, بھریہ, زمہ, داری, لگائی, اپوزیشن, کردار, کشی, اوراس, پربھونکناہےاپ, خوشامدگری, وچاپلوسی, سےاورکتنی, دولت, کماناچاہتےہیں, موٹرسائیکل, سےپیجارو, پراڈو, کےسفرمیں, ضمیرکی, لاش, سےاٹھتی, بدبواپ, ناک, نہیں, کرتی, نوٹ, سب, سےالتجاگزارش, ہےہمیں, فالوکریں, شکریہ]"
4,"[مراد, علی, شاہ, بھیس, ڈی, جی, ائی, ایس, ائی, حامد, میر]"
5,"[قابل, اعتبار, قاتل, اعتبار]"


Tf-IDF

In [284]:

# TF-IDF Vectorization

# Initialize the TF-IDF vectorizer
vectorizer  = TfidfVectorizer()

# Fit and transform the cleaned text
tfidf_matrix = vectorizer.fit_transform(data['lemmatized_text'])

# Get the words corresponding to the features
feature_names = vectorizer.get_feature_names_out()

# Create a DataFrame to store the TF-IDF scores
tfidf_scores = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Sum the scores for each term across all documents
sum_tfidf_scores = tfidf_scores.sum(axis=0)

# Sort the scores and get the top 10 terms
top_tfidf_terms = sum_tfidf_scores.nlargest(10)

# Display the top 10 words with the highest TF-IDF scores
print(top_tfidf_terms)

نہیں    534.109678
کہنا    435.896408
کرنا    405.821071
رہنا    402.511894
لینا    331.921184
پرنا    324.349406
میں     291.783109
دینا    289.974836
لگنا    261.155599
نا      231.480772
dtype: float64


Word2Vec

In [285]:
# Create the Word2Vec model
word2vec_model = Word2Vec(sentences=data['tokenized_text'], vector_size=100, window=5, min_count=1, workers=4)

# Save the model
word2vec_model.save("word2vec_urdu.model")

**N-grams Analysis**(Unigram, Bigram, and Trigram Analysis)

In [286]:
vectorizer1  = TfidfVectorizer()
# For Unigrams, Bigrams, and Trigrams
vectorizer1 = CountVectorizer(ngram_range=(1, 3))  # (1, 3) for Unigrams, Bigrams, and Trigrams
ngram_matrix = vectorizer1.fit_transform(data['lemmatized_text'])

# Convert to DataFrame
ngram_df = pd.DataFrame(ngram_matrix.toarray(), columns=vectorizer1.get_feature_names_out())

# Sum the counts for each n-gram across all documents
ngram_sums = ngram_df.sum(axis=0)

# Separate unigrams, bigrams, and trigrams
unigram_sums = ngram_sums[ngram_sums.index.str.split(' ').str.len() == 1]
bigram_sums = ngram_sums[ngram_sums.index.str.split(' ').str.len() == 2]
trigram_sums = ngram_sums[ngram_sums.index.str.split(' ').str.len() == 3]

# Get top 10 unigrams, bigrams, and trigrams
top_unigrams = unigram_sums.nlargest(10)
top_bigrams = bigram_sums.nlargest(10)
top_trigrams = trigram_sums.nlargest(10)

# Display the top 10 unigrams, bigrams, and trigrams with their frequencies
print("\nTop 10 Unigrams:")
print(top_unigrams)

print("\nTop 10 Bigrams:")
print(top_bigrams)

print("\nTop 10 Trigrams:")
print(top_trigrams)



Top 10 Unigrams:
نہیں    4442
کہنا    3903
کرنا    3529
رہنا    3118
پرنا    2760
لینا    2383
دینا    2054
میں     1992
لگنا    1584
والا    1481
dtype: int64

Top 10 Bigrams:
عمران خان     502
نواز شریف     449
ائی جینا      323
سندھ پولیس    299
پرنا تنقید    244
ارمی چیف      224
دیکھ نا       206
کرنا والا     195
کہنا کہنا     194
مر نا         192
dtype: int64

Top 10 Trigrams:
ائی جینا سندھ      118
پینا ٹی ائی        115
صلی اللہ علیہ       95
علیہ والہ وسلم      93
پینا ڈی ایم         88
پرنا تنقید کرنا     85
اللہ علیہ والہ      84
جزاک اللہ خیر       76
ائی جینا اغوا       73
فالو کرنا فالو      72
dtype: int64


**Sentiment Classification Model**

Model Building

In [287]:

# Split the data into features and target
X = tfidf_matrix  # You can also use ngram_matrix or features from Word2Vec
y = data['is_sarcastic']  # Assuming this is your sentiment label

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.7861437908496732
              precision    recall  f1-score   support

         0.0       0.78      0.78      0.78      1831
         1.0       0.80      0.79      0.79      1994

    accuracy                           0.79      3825
   macro avg       0.79      0.79      0.79      3825
weighted avg       0.79      0.79      0.79      3825



Prediction on new input

In [288]:
def predict_sentiment(new_input):
    # Preprocess the input
    processed_input = preprocess_text(new_input)
    processed_input = urdu_text_processing.lemmatize_sentence(processed_input)

    # Print processed input
    print(f"Processed input: {processed_input}")

    # Vectorize the processed input
    input_vectorized = vectorizer.transform([processed_input])  # Use transform() for new data

    # Make a prediction
    prediction = model.predict(input_vectorized)

    return prediction[0]


In [289]:
# Example usage
new_sentence = "بہت اچھا کام کیا، ہر سوال غلط دیا!"
predicted_sentiment = predict_sentiment(new_sentence)
if predicted_sentiment == 1:
    print("The sentiment is sarcastic")
else:
    print("The sentiment is not sarcastic.")


Processed input: بہت اچھا کام کیا ہرنا سوال غلط دیا
The sentiment is sarcastic
